# **Homework 1: COVID-19 Cases Prediction (Regression) -1**

Objectives:
* Solve a regression problem with deep neural networks (DNN).
* Understand basic DNN training tips.
* Familiarize yourself with PyTorch.

If you have any questions, please contact the TAs via TA hours, NTU COOL, or email to mlta-2022-spring@googlegroups.com

# Download data
If the Google Drive links below do not work, you can download data from [Kaggle](https://www.kaggle.com/c/ml2022spring-hw1/data), and upload data manually to the workspace.

In [1]:
!gdown --id '1kLSW_-cW2Huj7bh84YTdimGBOJaODiOS' --output covid.train.csv
!gdown --id '1iiI5qROrAhZn-o4FPqsE97bMzDEFvIdg' --output covid.test.csv

# Import packages

In [2]:
# Numerical Operations
import math
import numpy as np

# Reading/Writing Data
import pandas as pd
import os
import csv

# For Progress Bar
from tqdm import tqdm

# Pytorch
import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter

# Some Utility Functions

You do not need to modify this part.

In [3]:
def same_seed(seed): 
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def train_valid_split(data_set, valid_ratio, seed):
    '''Split provided training data into training set and validation set'''
    valid_set_size = int(valid_ratio * len(data_set)) 
    train_set_size = len(data_set) - valid_set_size
    train_set, valid_set = random_split(data_set, [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(seed))
    return np.array(train_set), np.array(valid_set)

def predict(test_loader, model, device):
    model.eval() # Set your model to evaluation mode.
    preds = []
    for x in tqdm(test_loader):
        x = x.to(device)                        
        with torch.no_grad():                   
            pred = model(x)                     
            preds.append(pred.detach().cpu())   
    preds = torch.cat(preds, dim=0).numpy()  
    return preds

# Dataset

In [4]:
class COVID19Dataset(Dataset):
    '''
    x: Features.
    y: Targets, if none, do prediction.
    '''
    def __init__(self, x, y=None):
        if y is None:
            self.y = y
        else:
            self.y = torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)
        
    def __getitem__(self, idx):
        if self.y is None:
            return self.x[idx]
        else:
            return self.x[idx], self.y[idx]

    def __len__(self):
        return len(self.x)

# Neural Network Model
Try out different model architectures by modifying the class below.

In [6]:
class My_Model(nn.Module):
    def __init__(self, input_dim):
        super(My_Model, self).__init__()
        # TODO: modify model's structure, be aware of dimensions. 
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.Dropout(0.4),
            nn.LeakyReLU(),
            nn.Linear(512, 1),
        )

    def forward(self, x):
        x = self.layers(x)
        x = x.squeeze(1) # (B, 1) -> (B)
        return x

# Feature Selection
Choose features you deem useful by modifying the function below.

In [7]:
def select_feat(train_data, valid_data, test_data, select_all=True, feature_selection_idx_list=None):
    '''Selects useful features to perform regression'''
    y_train, y_valid = train_data[:,-1], valid_data[:,-1]
    raw_x_train, raw_x_valid, raw_x_test = train_data[:,:-1], valid_data[:,:-1], test_data

    if not select_all and feature_selection_idx_list is not None:
        feat_idx = feature_selection_idx_list
    else:
        feat_idx = list(range(raw_x_train.shape[1]))
        
    return raw_x_train[:,feat_idx], raw_x_valid[:,feat_idx], raw_x_test[:,feat_idx], y_train, y_valid

# Training Loop

In [8]:
def trainer(train_loader, valid_loader, model, config, device):

    criterion = nn.MSELoss(reduction='mean') # Define your loss function, do not modify this.

    # Define your optimization algorithm. 
    # TODO: Please check https://pytorch.org/docs/stable/optim.html to get more available algorithms.
    # TODO: L2 regularization (optimizer(weight decay...) or implement by your self).
    optimizer = torch.optim.AdamW(model.parameters(), weight_decay=5e-5)  # 5e-5
#     optimizer = torch.optim.SGD(model.parameters(), lr=config['learning_rate'], momentum=0.5) 

    writer = SummaryWriter() # Writer of tensoboard.

    if not os.path.isdir('./models'):
        os.mkdir('./models') # Create directory of saving models.

    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0

    for epoch in range(n_epochs):
        model.train() # Set your model to train mode.
        loss_record = []

        # tqdm is a package to visualize your training progress.
        train_pbar = tqdm(train_loader, position=0, leave=True)

        for x, y in train_pbar:
            optimizer.zero_grad()               # Set gradient to zero.
            x, y = x.to(device), y.to(device)   # Move your data to device. 
            pred = model(x)             
            loss = criterion(pred, y)  # RMSE
            loss.backward()                     # Compute gradient(backpropagation).
            optimizer.step()                    # Update parameters.
            step += 1
            loss_record.append(loss.detach().item())
            
            # Display current epoch number and loss on tqdm progress bar.
            train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
            train_pbar.set_postfix({'loss': loss.detach().item()})

        mean_train_loss = sum(loss_record)/len(loss_record)
        writer.add_scalar('Loss/train', mean_train_loss, step)

        model.eval() # Set your model to evaluation mode.
        loss_record = []
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                loss = criterion(pred, y)

            loss_record.append(loss.item())
            
        mean_valid_loss = sum(loss_record)/len(loss_record)
        print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')
        writer.add_scalar('Loss/valid', mean_valid_loss, step)

        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), config['save_path']) # Save your best model
            print('Saving model with loss {:.3f}...'.format(best_loss))
            early_stop_count = 0
        else: 
            early_stop_count += 1

        if early_stop_count >= config['early_stop']:
            print('\nModel is not improving, so we halt the training session.')
            print(f'Best Loss: {best_loss}')
            return

# Configurations
`config` contains hyper-parameters for training and the path to save your model.

In [9]:
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.cuda.set_device(1) 
device = torch.device("cuda", 1)
print(device)

config = {
    'seed': 5201314,      # 5201314 Your seed number, you can pick your lucky number. :)
    'select_all': True,   # True Whether to use all features.
    'valid_ratio': 0.3, # 0.3   # validation_size = train_size * valid_ratio
    'n_epochs': 5000,     # 5000 Number of epochs.            
    'batch_size': 1024,  # 1024
    'learning_rate': 2e-4, # 2e-4             
    'early_stop': 300,   # 300 If model has not improved for this many consecutive epochs, stop training.     
    'save_path': './models/model_train_test_73_1.ckpt'  # Your model will be saved here.
}

cuda:1


# Dataloader
Read data from files and set up training, validation, and testing sets. You do not need to modify this part.

In [10]:
# Set seed for reproducibility
same_seed(config['seed'])


# train_data size: 2699 x 118 (id + 37 states + 16 features x 5 days) 
# test_data size: 1078 x 117 (without last day's positive rate)
train_data, test_data = pd.read_csv('./covid.train_test_1.csv').values, pd.read_csv('./covid.test_1.csv').values
train_data, valid_data = train_valid_split(train_data, config['valid_ratio'], config['seed'])

# Print out the data size before feature selection.
print(f""" -- Before feature selection --
train_data size: {train_data.shape} 
valid_data size: {valid_data.shape} 
test_data size: {test_data.shape}""")

# Select features
x_train, x_valid, x_test, y_train, y_valid = select_feat(train_data, valid_data, test_data, config['select_all'])

# Print out the data size after feature selection.
print(f""" -- After feature selection --
train_data size: {x_train.shape} 
valid_data size: {x_valid.shape} 
test_data size: {x_test.shape}""")

# Print out the number of features.
print(f'number of features: {x_train.shape[1]}')

train_dataset = COVID19Dataset(x_train, y_train)
valid_dataset = COVID19Dataset(x_valid, y_valid)
test_dataset = COVID19Dataset(x_test)
                                            
# Pytorch data loader loads pytorch dataset into batches.
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)

 -- Before feature selection --
train_data size: (9821, 10) 
valid_data size: (4209, 10) 
test_data size: (1078, 9)
 -- After feature selection --
train_data size: (9821, 9) 
valid_data size: (4209, 9) 
test_data size: (1078, 9)
number of features: 9


In [11]:
# print(relevant_features)

In [12]:
# train_data size: (2160, 118) 
# valid_data size: (539, 118) 
# test_data size: (1078, 117)
# number of features: 117

# Start training!

In [13]:
model = My_Model(input_dim=x_train.shape[1]).to(device) # put your model and data on the same computation device.
trainer(train_loader, valid_loader, model, config, device)

Epoch [1/5000]: 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 27.38it/s, loss=33]


Epoch [1/5000]: Train loss: 56.5911, Valid loss: 28.7503
Saving model with loss 28.750...


Epoch [2/5000]: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 218.75it/s, loss=14.2]


Epoch [2/5000]: Train loss: 27.6122, Valid loss: 6.0561
Saving model with loss 6.056...


Epoch [3/5000]: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 219.26it/s, loss=14.9]


...

Epoch [1225/5000]: Train loss: 1.2807, Valid loss: 1.0070


Epoch [1226/5000]: 100%|███████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 97.55it/s, loss=1.33]


Epoch [1226/5000]: Train loss: 1.2900, Valid loss: 1.0637


Epoch [1227/5000]: 100%|███████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 215.08it/s, loss=1.3]


Epoch [1227/5000]: Train loss: 1.2696, Valid loss: 1.0314

Model is not improving, so we halt the training session.
Best Loss: 0.9647051930427551


In [14]:
# Best Valid loss without FS: 2.4238
# Best Valid loss with FS: 1.1371
# Best Valid loss with Adam: 1.0121
# Best Valid loss with AdamW: 0.9102
# Current Best: 0.9029

# 0.8854

# Plot learning curves with `tensorboard` (optional)

`tensorboard` is a tool that allows you to visualize your training progress.

If this block does not display your learning curve, please wait for few minutes, and re-run this block. It might take some time to load your logging information. 

In [15]:
%reload_ext tensorboard
%tensorboard --logdir=./runs/

# Testing
The predictions of your model on testing set will be stored at `pred.csv`.

In [16]:
def save_pred(preds, file):
    ''' Save predictions to specified file '''
    with open(file, 'w') as fp:
        writer = csv.writer(fp)
        writer.writerow(['id', 'tested_positive'])
        for i, p in enumerate(preds):
            writer.writerow([i, p])

model = My_Model(input_dim=x_train.shape[1]).to(device)
model.load_state_dict(torch.load(config['save_path']))
preds = predict(test_loader, model, device) 
save_pred(preds, 'pred.csv')         

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 857.64it/s]


In [17]:
preds

array([ 8.298123 ,  9.0597315,  5.1944437, ..., 36.64625  , 36.292133 ,
       39.311195 ], dtype=float32)

In [18]:
# np.save('pred_arr_train_test_1', preds)

# Reference
This notebook uses code written by Heng-Jui Chang @ NTUEE (https://github.com/ga642381/ML2021-Spring/blob/main/HW01/HW01.ipynb)